# 분석 전 기본세팅
> 분석 전 기본세팅, 필요코드 (불러오기, 내보내기, random forest, catboost, cross validation ..)

- toc: true 
- badges: true
- comments: true
- categories: [Python]
- image: images/

# 파일

## 파일 불러오기

``` python
df = pd.read_csv("G:/내 드라이브/bb/cc/data.csv')
```

## 파일 내보내기

``` python
test.to_csv('test.csv', index = False)
```

- 해당 소스코드가 있는 곳에 파일이 내보내진다.

---

# 라이브러리

## 기본

In [4]:
import pandas as pd # pandas

import numpy as np # numpy

import matplotlib.pyplot as plt # matplotlib
import matplotlib

import seaborn as sns # seaborn

## 캣부스트

In [ ]:
from catboost import CatBoostRegressor # 캣부스트 회귀
from catboost import CatBoostClassifier # 캣부스트 분류